Make video from photos in directory, based on photos, determined by time of recording

Print date and time on top of video

Indicate frames where the bee was rewarded


Callin Switzer

11 Dec 2016




In [51]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

from datetime import datetime

In [330]:
%qtconsole

In [2]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [671]:
# adjust gamma
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [672]:
os.chdir(("/Users/callinswitzer/Dropbox/ExperSummer2016/"
            "BeeSonicationLearningWithAvery/"
             "BeeFrequencyLearning/"))

In [673]:
os.listdir(os.getcwd())

['.DS_Store',
 'BeeBlue2_28Nov2016_Hive4_high',
 'BeeBlue3_28Nov2016_Hive4_high',
 'BeeBlue4_28Nov2016_Hive4_high',
 'BeeBlue5_28Nov2016_Hive4_high',
 'BeeBlue6_28Nov2016_Hive4_high',
 'BeeBlue7_28Nov2016_Hive4_high',
 'Beeblue8_29Nov2016_Hive4_high',
 'BeeBlue_22Nov2016_Hive4_initial',
 'Beegold1_29Nov2016_Hive3_initial',
 'Beegoldred_02Dec2016_Hive4_initial',
 'Beegreen1_29Nov2016_Hive4_initial',
 'Beegreen2_02Dec2016_Hive4_initial',
 'Beelime1_12Dec2016_Hive5_initial',
 'BeeOrange3_28Nov2016_Hive3_low',
 'Beeorange4_30Nov2016_Hive3_low',
 'Beeorange5_30Nov2016_Hive3_low',
 'Beeorange6_02Dec2016_Hive3_high',
 'Beeorange7_09Dec2016_Hive3_high',
 'Beeorange8_09Dec2016_Hive3_high',
 'BeeOrange_28Nov2016_Hive3_initial',
 'BeeOrange_28Nov2016_Hive3_low',
 'BeePink_21Nov2016_Hive4_high',
 'BeePink_21Nov2016_Hive4_low',
 'Beepurple1_30Nov2016_Hive3_initial',
 'Beeredblue10_02Dec2016_Hive4_initial',
 'Beeredblue11_02Dec2016_Hive4_initial',
 'Beeredblue1_29Nov2016_Hive4_initial',
 'Beeredblue

In [674]:
beeTrialList =  [ f for f in os.listdir(".") if not f.startswith(r".") ]

In [680]:
trialNum = 50
CurrentTrialDir = os.path.join( os.getcwd() , beeTrialList[trialNum])
trialFolder = [ f for f in os.listdir(CurrentTrialDir) if not f.endswith(r".txt") and not f.startswith(r".") ]

trialFolderDir = os.path.join(CurrentTrialDir, trialFolder[0])
photoList = [p for p in os.listdir(trialFolderDir) if p.endswith(r".pgm")]

# remove background photo from photolist
photoList.remove('background.pgm')

In [681]:
# sort list by date created
creDate = [os.stat(os.path.join(trialFolderDir, c)).st_mtime for c in  photoList]
creD_time = [datetime.fromtimestamp(aa).strftime('%Y-%m-%d %H:%M:%S') for aa in creDate]

sortedPhotos = np.array(photoList)[np.argsort(np.array(creDate))]

In [693]:
# label rewards -- 
# HERE -- need to read in other metadata, because right now, it's also counting wingbeatsß
cc = 1
rewd = [ not 'bkg' in sortedPhotos[kk] for kk in range(len(sortedPhotos))]
rr = np.cumsum(rewd)

In [695]:
# make file with metadata from video
dd = pd.DataFrame({'photoName':sortedPhotos, 
                   'vidFrame':range(1, len(sortedPhotos) + 1), 
                  'dateModified':np.array(creD_time)[np.argsort(np.array(creDate))], 
                  'rewardNum':rr})
dd




,dateModified,photoName,rewardNum,vidFrame
0,2016-12-12 08:48:56,2016_12_12__08_48_56_618_bkg.pgm,0,1
1,2016-12-12 08:48:57,2016_12_12__08_48_57_305_bkg.pgm,0,2
2,2016-12-12 08:48:58,2016_12_12__08_48_58_027_bkg.pgm,0,3
3,2016-12-12 08:48:58,2016_12_12__08_48_58_727_bkg.pgm,0,4
4,2016-12-12 08:48:59,2016_12_12__08_48_59_007.pgm,1,5
5,2016-12-12 08:48:59,2016_12_12__08_48_59_903_bkg.pgm,1,6
6,2016-12-12 08:49:00,2016_12_12__08_49_00_603_bkg.pgm,1,7
7,2016-12-12 08:49:01,2016_12_12__08_49_01_303_bkg.pgm,1,8
8,2016-12-12 08:49:01,2016_12_12__08_49_01_997_bkg.pgm,1,9
9,2016-12-12 08:49:02,2016_12_12__08_49_02_697_bkg.pgm,1,10


In [638]:
photoPaths = [os.path.join(trialFolderDir, c) for c in  sortedPhotos]

# sorted photo paths

kk = 0

# change directory
newD = '/Users/callinswitzer/Desktop/testStack/'
if not os.path.isdir(newD):
    os.mkdir(newD)


In [639]:
rewCtr = 1

for kk in range(len(sortedPhotos)):
    img = cv2.imread(photoPaths[kk], 0)


    s1 =  sortedPhotos[kk].split(".")[0]
    try:
        s2 = s1.split("__")
        s2[0] = s2[0].replace('_', '-')
        s2[1] = s2[1][:8].replace('_', ':') + '.' + s2[1][9:12]
        s3 = s2[0] + " " + s2[1]
    except:
        s3 = "No Timestamp"

    plt.clf()
    io.imshow(img)
    plt.axis([0, img.shape[1], img.shape[0], 0])
    plt.axis('off')

    plt.gca().set_position([0,0,1,1])
    plt.gca().set_aspect('equal')
    plt.text(850, 60, s3)
    #if s2[1]
    if not "bkg" in s1 and not "background" in s1:
        plt.text(850, 90, "REWARD " + str(rewCtr))
        rewCtr += 1

    # save figure
    plt.savefig(newD + str(kk).zfill(4) + '.png')
    print str(kk) + ' of ' + str(len(sortedPhotos))





0 of 236
1 of 236
2 of 236
3 of 236
4 of 236
5 of 236
6 of 236
7 of 236
8 of 236
9 of 236
10 of 236
11 of 236
12 of 236
13 of 236
14 of 236
15 of 236
16 of 236
17 of 236
18 of 236
19 of 236
20 of 236
21 of 236
22 of 236
23 of 236
24 of 236
25 of 236
26 of 236
27 of 236
28 of 236
29 of 236
30 of 236
31 of 236
32 of 236
33 of 236
34 of 236
35 of 236
36 of 236
37 of 236
38 of 236
39 of 236
40 of 236
41 of 236
42 of 236
43 of 236
44 of 236
45 of 236
46 of 236
47 of 236
48 of 236
49 of 236
50 of 236
51 of 236
52 of 236
53 of 236
54 of 236
55 of 236
56 of 236
57 of 236
58 of 236
59 of 236
60 of 236
61 of 236
62 of 236
63 of 236
64 of 236
65 of 236
66 of 236
67 of 236
68 of 236
69 of 236
70 of 236
71 of 236
72 of 236
73 of 236
74 of 236
75 of 236
76 of 236
77 of 236
78 of 236
79 of 236
80 of 236
81 of 236
82 of 236
83 of 236
84 of 236
85 of 236
86 of 236
87 of 236
88 of 236
89 of 236
90 of 236
91 of 236
92 of 236
93 of 236
94 of 236
95 of 236
96 of 236
97 of 236
98 of 236
99 of 236
100 of 236

In [632]:
oldDir = os.getcwd()

In [640]:
os.chdir(newD)
# use ffmpeg to convert directory to video
stta = time.time()
!ffmpeg -start_number 0 -r 5 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y test.mp4
print time.time() - stta # ~ 1.4 seconds

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '%04d.png':
  Duration: 00:00:09.44, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgba(pc), 800x600 [SAR 3937:3937 DAR 4:3], 25 fps, 25 tbr, 25 tbn, 25 tbc
[libx264

In [641]:
# delete all png files in directory
filelist = [ f for f in os.listdir(newD) if f.endswith(".png") ]

for f in filelist:
    os.remove(f)

In [111]:
creD_time = [datetime.fromtimestamp(aa).strftime('%Y-%m-%d %H:%M:%S') for aa in creDate]
creD_time

['2016-11-28 11:08:33',
 '2016-11-28 11:09:16',
 '2016-11-28 11:09:43',
 '2016-11-28 11:09:43',
 '2016-11-28 11:09:44',
 '2016-11-28 11:09:44',
 '2016-11-28 11:09:49',
 '2016-11-28 11:10:01',
 '2016-11-28 11:10:02',
 '2016-11-28 11:10:03',
 '2016-11-28 11:10:03',
 '2016-11-28 11:08:20']

In [109]:
np.argsort(np.array(creDate))

array([11,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [112]:
dd = pd.DataFrame({'file': np.array(photoList), 'time': np.array(creDate), 'time2':creD_time})
dd

,file,time,time2
0,2016_11_28__11_08_33_618.pgm,1.480349e+09,2016-11-28 11:08:33
1,2016_11_28__11_09_16_734.pgm,1.480349e+09,2016-11-28 11:09:16
2,2016_11_28__11_09_43_099.pgm,1.480349e+09,2016-11-28 11:09:43
3,2016_11_28__11_09_43_694.pgm,1.480349e+09,2016-11-28 11:09:43
4,2016_11_28__11_09_44_424.pgm,1.480349e+09,2016-11-28 11:09:44
5,2016_11_28__11_09_44_931.pgm,1.480349e+09,2016-11-28 11:09:44
6,2016_11_28__11_09_49_468.pgm,1.480349e+09,2016-11-28 11:09:49
7,2016_11_28__11_10_01_000.pgm,1.480349e+09,2016-11-28 11:10:01
8,2016_11_28__11_10_02_970.pgm,1.480349e+09,2016-11-28 11:10:02
9,2016_11_28__11_10_03_388.pgm,1.480349e+09,2016-11-28 11:10:03


In [113]:
dd.ix[np.argsort(np.array(creDate)), :]

,file,time,time2
11,background.pgm,1.480349e+09,2016-11-28 11:08:20
0,2016_11_28__11_08_33_618.pgm,1.480349e+09,2016-11-28 11:08:33
1,2016_11_28__11_09_16_734.pgm,1.480349e+09,2016-11-28 11:09:16
2,2016_11_28__11_09_43_099.pgm,1.480349e+09,2016-11-28 11:09:43
3,2016_11_28__11_09_43_694.pgm,1.480349e+09,2016-11-28 11:09:43
4,2016_11_28__11_09_44_424.pgm,1.480349e+09,2016-11-28 11:09:44
5,2016_11_28__11_09_44_931.pgm,1.480349e+09,2016-11-28 11:09:44
6,2016_11_28__11_09_49_468.pgm,1.480349e+09,2016-11-28 11:09:49
7,2016_11_28__11_10_01_000.pgm,1.480349e+09,2016-11-28 11:10:01
8,2016_11_28__11_10_02_970.pgm,1.480349e+09,2016-11-28 11:10:02


In [38]:
sorted(photoList, reverse = False)

['2016_11_28__11_08_33_618.pgm',
 '2016_11_28__11_09_16_734.pgm',
 '2016_11_28__11_09_43_099.pgm',
 '2016_11_28__11_09_43_694.pgm',
 '2016_11_28__11_09_44_424.pgm',
 '2016_11_28__11_09_44_931.pgm',
 '2016_11_28__11_09_49_468.pgm',
 '2016_11_28__11_10_01_000.pgm',
 '2016_11_28__11_10_02_970.pgm',
 '2016_11_28__11_10_03_388.pgm',
 '2016_11_28__11_10_03_987.pgm',
 'background.pgm']

In [96]:
x = np.array([3, 1, 2])
ind = np.argsort(x)

In [97]:
x[ind]

array([1, 2, 3])

In [98]:
ind

array([1, 2, 0])

In [ ]:
filelist = [ f for f in os.listdir(".") if f.endswith(".png") and not f.startwith(r".") ]